In [1]:
import Analysis
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


In [3]:
def search(data, t):
    seq = data
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

In [4]:
from matplotlib.gridspec import GridSpec
mpl.rcParams.update({'font.size': 8})
gs = GridSpec(1000,1000)

In [5]:
def init_plot(i):
    i.plot()
    plt.xlim([min(i.x),max(i.x)])
    plt.ylim([min(i.y)-0.05,max(i.y)+0.05])
    
def stitch(i,x):
    i.fits[1].plot_select('b',['k','r'],[2,2],i.fits[1].pos_left,x)
    i.fits[0].plot_select('b',['r','#00ff00'],[2,2],x,i.fits[0].pos_right)
    
def set_axis(sweep,x_min,x_max):
    plt.xlim(x_min,x_max)
    x = np.linspace(x_min,x_max)
    yl,yr = sweep.fits[j].yValue(x)
    y_min = np.min([yl,yr])
    y_max = np.max([yl,yr])
    l,h = t(y_min,y_max)
    plt.ylim(l,h)
    plt.yticks([y_min,y_max],(round(y_min,1),round(y_max,1)))
    x_low, x_high = np.percentile(x,[25,75])
    plt.xticks([int(x_low),int(x_high)])


In [6]:
plt.rc('text', usetex=True) 

# Helvetica 
# set serif, too. Otherwise setting to times and then 
# Helvetica causes an error. 
font = {'family':'sans-serif','sans-serif':['Helvetica'], 
        'serif':['cm10']} 
preamble  = r""" 
               \usepackage{color} 
               \usepackage[tx]{sfmath} 
               \usepackage{helvet} 
            """ 


# Unicode for Tex 
#preamble =  r"""\usepackage[utf8]{inputenc}""" + preamble 
# inputenc should be set automatically 
plt.rcParams['text.latex.unicode']=True 

#print font, preamble 
plt.rc('font',**font) 
plt.rcParams['text.latex.preamble'] = preamble
mpl.rcParams.update({'font.size': 11})

In [7]:
def t(y_min,y_max):
    diff = y_max-y_min
    p = float(diff)*0.35
    return y_min-p,y_max+p

In [8]:
run14 = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_strong.npy'), Analysis.bval2014, np.load('2014bf_new_prune_edit.npy'))
run11 = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_strong_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))

soft11 = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))
soft14 = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_new.npy'), Analysis.bval2014, np.load('2014bf_new_prune_edit.npy'))

i14 = run14.prune_data
i11 = run11.prune_data

In [9]:
se14 = np.load('2014se_new_sign.npy')
se11 = np.load('2011se_new_sign.npy')
#use this to plot slips:
#for k in se11[n][np.abs(se11[n][:,2] - i11[n].fits[j].delta_phi)<0.01]:
    #ax5.axvline(k[0],color='k',lw=2)

In [10]:
pi4_14 = [[0,0],[0,2],[0,3],[2,1],[2,2],[3,3],[3,4],[4,1],[5,0],[7,2],[8,0],[8,3],[9,0]]
pi2_14 = [[3,2],[3,1]] 
pi34_14 = [[2,0],[2,3],[3,0],[8,1]]
pi_14 = [[0,1],[4,0],[5,1],[6,0],[7,1],[8,2],[9,1],[10,0],[11,0],[13,0],[13,1]]
pi74_14 = [[5,2]]
#------------------------------------------
pi4_11 = [[0,0],[5,0],[9,1],[11,0],[14,0],[19,1]]
pi2_11 = [[2,1],[4,0],[12,1],[15,0],[19,0]]
pi34_11 = [[0,1],[1,1],[10,0],[13,0]]
pi_11 = [[1,0],[2,0],[3,0],[4,1],[5,1],[6,0],[7,0],[8,0],[9,2],[11,1],[12,0],[14,1],[15,1],[16,0],[17,0],[19,2],[21,0]]
#No pi7/4 for 2011

In [443]:
#PI/4 Events
mpl.rcParams.update({'font.size': 11})
L = len(pi4_14) + len(pi4_11)
fig, ax = plt.subplots(L, 1, figsize=(5,12))

for i in range(len(pi4_14)):
    plt.sca(ax[i])
    run14.prune_data[pi4_14[i][0]].plot_fit(pi4_14[i][1],False)
    
for i in range(len(pi4_11)):
    plt.sca(ax[i+len(pi4_14)])
    run11.prune_data[pi4_11[i][0]].plot_fit(pi4_11[i][1],False)

In [11]:
def plotGrid(run,se,start,fit_list,k,x_low,x_high,colspan,rowspan,xy=(-0.05, 1.02),lw=1.2,linew=2):
    plt.subplot2grid((100,100), start, colspan=colspan, rowspan=rowspan)
    j = fit_list[k][1]
    n = fit_list[k][0]
    run.prune_data[n].plot_fit(j,linewidth=linew)
    axes(run.prune_data[n],j,x_low,x_high)
    for k in se[n][np.abs(se[n][:,2] - (run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi%2)<0.001]:
        plt.axvline(k[0],color='k',lw=lw)
    plt.annotate('{:.4f} T, $\Delta\\theta=$ {:.3f}'.format(run.prune_data[n].b_val,round(((run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi)%2,3)), xy, xycoords="axes fraction")
    
def axes(sweep,j,x_min,x_max):
    plt.xlim(x_min,x_max)
    x = np.linspace(x_min,x_max)
    yl,yr = sweep.fits[j].yValue(x)
    y_min = np.min([yl,yr])
    y_max = np.max([yl,yr])
    l,h = t(y_min,y_max)
    plt.ylim(l,h)
    plt.yticks([y_min,y_max],(round(y_min,1),round(y_max,1)))
    x_low, x_high = np.percentile(x,[25,75])
    plt.xticks([int(x_low),int(x_high)])
    ax = plt.gca()
    ax.tick_params('y',pad=2)
    ax.tick_params('x',pad=3)
    
    
def newplotGrid(bounds,colspan,rowspan,xy=(-0.05, 1.02)):
    run,se,fit_list,k,x_low,x_high = bounds
    j = fit_list[k][1]
    n = fit_list[k][0]
    run.prune_data[n].plot_fit(j)
    axes(run.prune_data[n],j,x_low,x_high)
    for k in se[n][np.abs(se[n][:,2] - (run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi%2)<0.001]:
        plt.axvline(k[0],color='k',lw=2)
    plt.annotate('{} T'.format(run.prune_data[n].b_val), xy, xycoords="axes fraction")

In [12]:
pi4_14_bounds = [[run14,se14,pi4_14,0,-11,-7],[run14,se14,pi4_14,1,-21,-16],
       [run14,se14,pi4_14,2,-23.67,-19.6],[run14,se14,pi4_14,3,-19.48,-15.83],
       [run14,se14,pi4_14,4,-21,-18.66],[soft14,se14,pi4_14,5,-21.6,-18.4],
       [soft14,se14,pi4_14,6,-27,-20.99],[soft14,se14,pi4_14,7,-23.14,-19.58],
       [run14,se14,pi4_14,8,-14.16,-7.65],[run14,se14,pi4_14,9,-26.29,-19.04],
       [run14,se14,pi4_14,10,-31.0675,-25.2242],[soft14,se14,pi4_14,11,-25.6274,-18.36],
       [run14,se14,pi4_14,12,-25.78,-19.49]]

In [19]:
height = 7
width = 2
row_extra = 2
row_len = (100-row_extra*(height-1))/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (2.76/(col_len/100.),1.7/(row_len/100.)))
for i in range(13):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi4_14_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len)
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 6:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05, -0.15)
        

In [91]:
fig.savefig('2014 025Pi Supplemental v3',dpi=1000)

In [13]:
pi4_11_bounds = [[soft11,se11,pi4_11,0,-31.9,-23.479],[soft11,se11,pi4_11,1,-31.84,-26.8],
       [soft11,se11,pi4_11,2,-32,-14.24],[soft11,se11,pi4_11,3,-31.53,-17.82],
       [soft11,se11,pi4_11,4,-33.6,-24.22],[soft11,se11,pi4_11,5,-33.5,-28]]

In [145]:
height = 4
width = 2
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (6,1.7/(row_len/100.)))
for i in range(6):
    hold = pi4_11_bounds[i]
    current = (i/2*(row_extra+row_len),i%2*(col_extra+col_len))
    if i == 2:
        current = (1*(row_extra+row_len),0)
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.025, 1.02))
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.065, 0)
        continue
    if i == 3:
        current = (2*(row_extra+row_len),0)
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.025, 1.02))
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(0.5, -1.25)
        continue
    if i > 3:
        current = (3*(row_extra+row_len),i%2*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len)

In [146]:
fig.savefig('2011 025Pi Supplemental v3',dpi=1000)

In [14]:
d140 = np.load('2014_0.npy')
d144 = np.load('2014_4.npy')

In [15]:
pi_14_bounds = [[soft14,se14,pi_14,0,-19.48,-9.83],[run14,se14,pi_14,1,-18.18,-7.05],
       [run14,se14,pi_14,2,-21.14,-8.78],[run14,se14,pi_14,3,-16.54,-9],
       [soft14,se14,pi_14,4,-17.8,-10.38],[run14,se14,pi_14,5,-15.18,-7.22],
       [soft14,se14,pi_14,6,-17.8,-9.72],[soft14,se14,pi_14,7,-20.02,-9.20],
       [run14,se14,pi_14,8,-20.4,-8.65],[run14,se14,pi_14,9,-19.32,-7.76],
       [run14,se14,pi_14,10,-27.51,-17.93]]

In [147]:
height = 5
width = 2
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (6,1.7/(row_len/100.)))
for i in range(10):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_14_bounds[i]
    if i == 0:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d140[0],d140[1])
        run14.prune_data[0].fits[1].plot(False)
        axes(run14.prune_data[0],1,hold[-2],hold[-1])
        for k in se14[0][np.abs(se14[0][:,2] - (run14.prune_data[0].fits[1].phase_r-run14.prune_data[0].fits[1].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=2)
        plt.annotate('{} T'.format(run14.prune_data[0].b_val), xy=(-0.05, 1.02), xycoords="axes fraction")
        continue
    if i == 1:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d144[0],d144[1])
        run14.prune_data[4].fits[0].plot(False)
        axes(run14.prune_data[4],0,hold[-2],hold[-1])
        for k in se14[4][np.abs(se14[4][:,2] - (run14.prune_data[4].fits[0].phase_r-run14.prune_data[4].fits[0].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=2)
        plt.annotate('{} T'.format(run14.prune_data[4].b_val), xy=(-0.05, 1.02), xycoords="axes fraction")
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len)
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 4:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05, -0.15)
        

In [98]:
fig.savefig('2014 Pi Supplemental v3',dpi=1000)

In [16]:
pi_11_bounds = [[soft11,se11,pi_11,0,-26.34,-18.02],[run11,se11,pi_11,1,-38.84,-29.39],
                [run11,se11,pi_11,2,-41.13,-29.99],[soft11,se11,pi_11,3,-42.59,-31.06],
                [run11,se11,pi_11,4,-41.27,-30.44],[soft11,se11,pi_11,5,-41,-30],
                [soft11,se11,pi_11,6,-40,-29.82],[run11,se11,pi_11,7,-39.23,-28.47],
                [run11,se11,pi_11,8,-39.9,-29.56],[run11,se11,pi_11,9,-38.2,-29.144],
                [run11,se11,pi_11,10,-42.07,-31.46],[run11,se11,pi_11,11,-41.87,-29.9],
                [run11,se11,pi_11,12,-41.29,-30.3],[run11,se11,pi_11,13,-40.29,-30.86],
                [run11,se11,pi_11,14,-41.36,-32.15],[run11,se11,pi_11,15,-41.33,-31.54],
                [run11,se11,pi_11,16,-40.7672,-30.3]]

In [104]:
height = 5
width = 2
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (6,1.7/(row_len/100.)))
for i in range(10):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_11_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len)
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 4:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05, -0.15)
        

In [100]:
fig.savefig('2011 Pi Supplemental 1_2 v3',dpi=1000)

In [130]:
height = 4
width = 2
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (6,1.7/(row_len/100.)))
for i in range(7):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_11_bounds[i+10]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len)
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.135, 1.1)
    if i == 3:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05, -0.15)
        

In [120]:
fig.savefig('2011 Pi Supplemental 2_2 v3',dpi=1000)

In [17]:
pi2_bounds = [[soft14,se14,pi2_14,0,-20,-15],[soft14,se14,pi2_14,1,-16.2,-10.3],
                [run11,se11,pi2_11,0,-42.48,-35.65],[soft11,se11,pi2_11,1,-35.46,-25.65],
                [soft11,se11,pi2_11,2,-42.47,-36.3],[soft11,se11,pi2_11,3,-31.59,-23.59],
                [run11,se11,pi2_11,4,-26.74,-19.2]]

In [121]:
height = 4
width = 2
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (6,1.7/(row_len/100.)))
for i in range(7):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi2_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len)
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.135, 1.1)
    if i == 3:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05, -0.15)

In [122]:
fig.savefig('05Pi Supplemental v3',dpi=1000)

In [18]:
pi34_bounds = [[run14,se14,pi34_14,0,-13.17,-4.05],[soft14,se14,pi34_14,1,-28.02,-21.7],
               [run14,se14,pi34_14,2,-12.9,-4.06],[soft14,se14,pi34_14,3,-12.04,-4.05],
               [run11,se11,pi34_11,0,-41.1,-34.2],[soft11,se11,pi34_11,1,-40.5,-31.7],
               [run11,se11,pi34_11,2,-37.159,-29],[run11,se11,pi34_11,3,-39.48,-30.16]]

In [18]:
height = 4
width = 2
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (6,1.7/(row_len/100.)))
for i in range(8):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi34_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len)
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.135, 1.1)
    if i == 3:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05, -0.15)

In [125]:
fig.savefig('075Pi Supplemental v3',dpi=1000)

In [197]:
height = 5
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(13):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi4_14_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 9:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)       

In [198]:
fig.savefig('2014 025Pi Supplemental v4',dpi=1000)

In [248]:
height = 3
width = 3
row_extra = 5
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(6):
    hold = pi4_11_bounds[i]
    current = (i/2*(row_extra+row_len),i%2*(col_extra+col_len))
    if i == 2:
        current = (1*(row_extra+row_len),0)
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.07, 1.02),lw=1.2)
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        continue
    if i == 3:
        current = (2*(row_extra+row_len),0)
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.07, 1.02),lw=1.2)
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(0.8, -0.15)
        continue
    if i > 3:
        current = (i%4*(row_extra+row_len),i/2*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)

In [237]:
fig.savefig('2011 025Pi Supplemental v4',dpi=1000)

In [325]:
height = 7
width = 3
row_extra = 2
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi4_bounds = pi4_14_bounds
pi4_bounds.extend(pi4_11_bounds)
for i in range(0,19):
    hold = pi4_bounds[i]
    if i > 9:
        i += 2
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    if i == 4:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 14:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
    if i == 17:
        current = (3*(row_extra+row_len),1*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.07, 1.02),lw=1.2)
        continue
    if i == 18:
        current = (4*(row_extra+row_len),1*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.07, 1.02),lw=1.2)
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)

In [326]:
fig.savefig('025Pi Supplemental v4',dpi=1000)

In [291]:
height = 4
width = 3
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(11):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_14_bounds[i]
    if i == 0:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d140[0],d140[1])
        run14.prune_data[0].fits[1].plot(False)
        axes(run14.prune_data[0],1,hold[-2],hold[-1])
        for k in se14[0][np.abs(se14[0][:,2] - (run14.prune_data[0].fits[1].phase_r-run14.prune_data[0].fits[1].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[0].b_val), xy=(-0.15, 1.02), xycoords="axes fraction")
        continue
    if i == 1:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d144[0],d144[1])
        run14.prune_data[4].fits[0].plot(False)
        axes(run14.prune_data[4],0,hold[-2],hold[-1])
        for k in se14[4][np.abs(se14[4][:,2] - (run14.prune_data[4].fits[0].phase_r-run14.prune_data[4].fits[0].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[4].b_val), xy=(-0.15, 1.02), xycoords="axes fraction")
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 7:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)    

In [292]:
fig.savefig('2014 Pi Supplemental v4',dpi=1000)

In [273]:
height = 6
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(17):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_11_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 11:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)

In [274]:
fig.savefig('2011 Pi Supplemental v4',dpi=1000)

In [294]:
height = 7
width = 3
row_extra = 2
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi_bounds = pi_14_bounds
pi_bounds.extend(pi_11_bounds)
for i in range(21):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_bounds[i]
    if i == 0:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d140[0],d140[1])
        run14.prune_data[0].fits[1].plot(False)
        axes(run14.prune_data[0],1,hold[-2],hold[-1])
        for k in se14[0][np.abs(se14[0][:,2] - (run14.prune_data[0].fits[1].phase_r-run14.prune_data[0].fits[1].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[0].b_val), xy=(-0.15, 1.02), xycoords="axes fraction")
        continue
    if i == 1:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d144[0],d144[1])
        run14.prune_data[4].fits[0].plot(False)
        axes(run14.prune_data[4],0,hold[-2],hold[-1])
        for k in se14[4][np.abs(se14[4][:,2] - (run14.prune_data[4].fits[0].phase_r-run14.prune_data[4].fits[0].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[4].b_val), xy=(-0.15, 1.02), xycoords="axes fraction")
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)    

In [295]:
fig.savefig('Pi Supplemental v4 1_2',dpi=1000)

In [307]:
height = 3
width = 3
row_extra = 5
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi_bounds = pi_14_bounds
pi_bounds.extend(pi_11_bounds)
for i in range(21,28):
    hold = pi_bounds[i]
    i -= 21
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)
    if i == 1:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)    

In [308]:
fig.savefig('Pi Supplemental v4 2_2',dpi=1000)

In [338]:
height = 3
width = 3
row_extra = 5
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(7):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi2_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)
    if i == 1:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)

    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)


In [339]:
fig.savefig('05Pi Supplemental v4',dpi=1000)

In [344]:
height = 3
width = 3
row_extra = 5
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(8):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi34_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.15, 1.02),lw=1.2)
    if i == 1:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)

In [341]:
fig.savefig('075Pi Supplemental v4',dpi=1000)

In [347]:
#7PI/4Events
fig = plt.figure(figsize = (1.0,1.85))
d = 7
plt.subplot2grid((100,100), (0,d), colspan=100-d, rowspan=83)

i = 0
k = 0
j = pi74_14[k][1]
n = pi74_14[k][0]
run14.prune_data[n].plot_fit(j)
set_axis(run14.prune_data[n],-30.37,-16.645)
plt.xticks([-30,-25,-20])
plt.axvline(-24.91,color='k',lw=1.2)
plt.axvline(-22.39,color='k',lw=1.2)
plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
print('{} T'.format(run14.prune_data[n].b_val))

5.582 T


In [348]:
fig.savefig('175Pi Supplemental v4',dpi=1000)

In [51]:
plt.rc('text', usetex=True) 

# Helvetica 
# set serif, too. Otherwise setting to times and then 
# Helvetica causes an error. 
font = {'family':'sans-serif','sans-serif':['Helvetica'], 
        'serif':['cm10']} 
preamble  = r""" 
               \usepackage{color} 
               \usepackage[tx]{sfmath} 
               \usepackage{helvet} 
            """ 


# Unicode for Tex 
#preamble =  r"""\usepackage[utf8]{inputenc}""" + preamble 
# inputenc should be set automatically 
plt.rcParams['text.latex.unicode']=True 

#print font, preamble 
plt.rc('font',**font) 
plt.rcParams['text.latex.preamble'] = preamble
mpl.rcParams.update({'font.size': 11})

In [185]:
height = 10
width = 3
row_extra = 2
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi_bounds = list(pi_11_bounds)
pi_bounds.extend(pi_14_bounds)
for i in range(len(pi_bounds)):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_bounds[i]
    if i == 17:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d140[0],d140[1])
        run14.prune_data[0].fits[1].plot(False)
        axes(run14.prune_data[0],1,hold[-2],hold[-1])
        for k in se14[0][np.abs(se14[0][:,2] - (run14.prune_data[0].fits[1].phase_r-run14.prune_data[0].fits[1].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[0].b_val), xy=(-0.05, 1.03), xycoords="axes fraction")
        continue
    if i == 18:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d144[0],d144[1])
        run14.prune_data[4].fits[0].plot(False)
        axes(run14.prune_data[4],0,hold[-2],hold[-1])
        for k in se14[4][np.abs(se14[4][:,2] - (run14.prune_data[4].fits[0].phase_r-run14.prune_data[4].fits[0].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[4].b_val), xy=(-0.05, 1.03), xycoords="axes fraction")
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 5:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 19:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)    

In [60]:
fig.savefig('Pi Supplemental v5',dpi=1000)

In [108]:
height = 7
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi4_bounds = list(pi4_14_bounds)
pi4_bounds.extend(pi4_11_bounds)
i = -1
for j in [3,6,1,5,12,15,16,9,14,2,8,13,10,0,4,11,17,18,7]:
    i+=1
    hold = pi4_bounds[j]
    if i == 12:
        i += 2
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    if i == 4:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    
    if i == 5:
        current = (5*(row_extra+row_len),0*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.022, 1.03),lw=1.2)
        continue
    if i == 6:
        current = (6*(row_extra+row_len),0*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.022, 1.03),lw=1.2)
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 20:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.65,-0.3)

In [97]:
fig.savefig('025Pi Supplemental v5',dpi=1000)

In [111]:
height = 3
width = 3
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(7):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi2_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 1:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)

    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)


In [112]:
fig.savefig('05Pi Supplemental v5',dpi=1000)

In [154]:
height = 3
width = 3
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
for i in range(8):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi34_bounds[i]
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 1:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)

In [152]:
fig.savefig('075Pi Supplemental v5',dpi=1000)

In [158]:
#7PI/4Events
height = 1
width = 1
row_extra = 20
row_len = (100-row_extra*(height))/height
col_extra = 5
col_len = (100-col_extra*(width))/width
fig = plt.figure(figsize = (3/(col_len/100.),1.69/(row_len/100.)))
hold = [run14,se14,pi74_14,0,-30.37,-16.645]
current = (0,col_extra)
plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.03, 1.03),lw=1.2)
plt.axvline(-24.91,color='k',lw=1.2)
plt.axvline(-22.39,color='k',lw=1.2)
plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
print('{} T'.format(run14.prune_data[n].b_val))

5.582 T


In [159]:
fig.savefig('175Pi Supplemental v5',dpi=1000)

In [37]:
height = 7
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi4_bounds = list(pi4_14_bounds)
pi4_bounds.extend(pi4_11_bounds)
i = -1
for j in [3,9,10,6,1,0,2,4,5,8,11,12,13,14,17,18,15,7,16]:
    i+=1
    hold = pi4_bounds[j]
    if i == 17:
        i += 1
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    if i == 10:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    
    if i == 16:
        current = (5*(row_extra+row_len),1*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.022, 1.03),lw=1.2)
        continue
    if i == 19:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.75,-0.3)
    if i == 19:
        current = (6*(row_extra+row_len),1*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.022, 1.03),lw=1.2)
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    

In [209]:
fig.savefig('025Pi Supplemental v6',dpi=1000)

In [210]:
height = 7
width = 4
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi_bounds = list(pi_11_bounds)
pi_bounds.extend(pi_14_bounds)
pi_bounds[1],pi_bounds[13] = pi_bounds[13],pi_bounds[1]
for i in range(len(pi_bounds)):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    hold = pi_bounds[i]
    if i == 17:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d140[0],d140[1])
        run14.prune_data[0].fits[1].plot(False)
        axes(run14.prune_data[0],1,hold[-2],hold[-1])
        for k in se14[0][np.abs(se14[0][:,2] - (run14.prune_data[0].fits[1].phase_r-run14.prune_data[0].fits[1].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[0].b_val), xy=(-0.05, 1.03), xycoords="axes fraction")
        continue
    if i == 18:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d144[0],d144[1])
        run14.prune_data[4].fits[0].plot(False)
        axes(run14.prune_data[4],0,hold[-2],hold[-1])
        for k in se14[4][np.abs(se14[4][:,2] - (run14.prune_data[4].fits[0].phase_r-run14.prune_data[4].fits[0].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[4].b_val), xy=(-0.05, 1.03), xycoords="axes fraction")
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.15,-0.3)

In [211]:
fig.savefig('Pi Supplemental v6-4',dpi=1000)

In [223]:
height = 10
width = 3
row_extra = 2
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi_bounds = list(pi_11_bounds)
pi_bounds.extend(pi_14_bounds)
pi_bounds[1],pi_bounds[8] = pi_bounds[8],pi_bounds[1]
pi_bounds[10],pi_bounds[15] = pi_bounds[15],pi_bounds[10]
temp = list(pi_bounds[9])
del pi_bounds[9]
pi_bounds.append(temp)
for i in range(len(pi_bounds)):
    hold = pi_bounds[i]
    if i > 8:
        i += 1
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    
    if i == 17:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d140[0],d140[1])
        run14.prune_data[0].fits[1].plot(False)
        axes(run14.prune_data[0],1,hold[-2],hold[-1])
        for k in se14[0][np.abs(se14[0][:,2] - (run14.prune_data[0].fits[1].phase_r-run14.prune_data[0].fits[1].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[0].b_val), xy=(-0.05, 1.03), xycoords="axes fraction")
        continue
    if i == 18:
        plt.subplot2grid((100,100), current, colspan = col_len,rowspan= row_len)
        plt.plot(d144[0],d144[1])
        run14.prune_data[4].fits[0].plot(False)
        axes(run14.prune_data[4],0,hold[-2],hold[-1])
        for k in se14[4][np.abs(se14[4][:,2] - (run14.prune_data[4].fits[0].phase_r-run14.prune_data[4].fits[0].phase_l)/np.pi%2)<0.001]:
            plt.axvline(k[0],color='k',lw=1.2)
        plt.annotate('{} T'.format(run14.prune_data[4].b_val), xy=(-0.05, 1.03), xycoords="axes fraction")
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 5:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 19:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)

In [215]:
fig.savefig('Pi Supplemental v6-3',dpi=1000)

In [238]:
height = 3
width = 3
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
i = 0
for j in [6,3,0,4,1,2,5]:
    hold = pi2_bounds[j]
    if i == 6:
        i += 1
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)

    if i == 7:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
    i += 1


In [222]:
fig.savefig('05Pi Supplemental v6',dpi=1000)

In [28]:
height = 3
width = 3
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
z = (100-2*col_len-col_extra)/2
i = 0
for j in [5,2,6,0,4,7,3,1]:
    hold = pi34_bounds[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    if i > 5:
        current = (i/width*(row_extra+row_len),z+i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 6:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.1,-0.23)
    i += 1

In [241]:
fig.savefig('075Pi Supplemental v6',dpi=1000)

In [251]:
#7PI/4Events
height = 1
width = 1
row_extra = 20
row_len = (100-row_extra*(height))/height
col_extra = 5
col_len = (100-col_extra*(width))/width
fig = plt.figure(figsize = (3/(col_len/100.),1.69/(row_len/100.)))
hold = [run14,se14,pi74_14,0,-30.37,-16.645]
current = (0,col_extra)
plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.03, 1.03),lw=1.2)
#plt.axvline(-24.91,color='k',lw=1.2)
#plt.axvline(-22.39,color='k',lw=1.2)
plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)

In [252]:
fig.savefig('175Pi Supplemental v6',dpi=1000)

In [250]:
se14[5][-1][0] = -22.39

In [304]:
#Fits for S1 and S2
height = 1
width = 1
row_extra = 10
row_len = (100-row_extra*(height))/height
col_extra = 5
col_len = (100-col_extra*(width))/width
fig = plt.figure(figsize = (2.8/(col_len/100.),1.69/(row_len/100.)))
x_low = -20
x_high = -7.69239
plt.subplot2grid((100,100), (0,col_extra), colspan=col_len, rowspan=row_len)
j = 1
n = 9
run = run14
run.prune_data[n].plot_fit(j,linewidth=2)
axes(run.prune_data[n],j,x_low,x_high)
plt.axvline(-13.8462,ls='--',color='k',lw=1.2)
plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)

In [305]:
fig.savefig('S1 v1',dpi=1000)

In [280]:
#Fits for S1 and S2
height = 1
width = 1
row_extra = 20
row_len = (100-row_extra*(height))/height
col_extra = 5
col_len = (100-col_extra*(width))/width
fig = plt.figure(figsize = (2.8/(col_len/100.),1.69/(row_len/100.)))
x_low = -39.3896
x_high = -31.3896
plt.subplot2grid((100,100), (0,col_extra), colspan=col_len, rowspan=row_len)
j = 2
n = 19
run = run11
run.prune_data[n].plot_fit(j,linewidth=2)
axes(run.prune_data[n],j,x_low,x_high)
plt.axvline(-35.3896,ls='--',color='k',lw=1.2)
plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=12)

In [281]:
fig.savefig('S1 v2',dpi=1000)

In [287]:
#Fits for S1 and S2
height = 1
width = 1
row_extra = 20
row_len = (100-row_extra*(height))/height
col_extra = 5
col_len = (100-col_extra*(width))/width
fig = plt.figure(figsize = (2.8/(col_len/100.),1.69/(row_len/100.)))
x_low = -37.5872
x_high = -30.35
plt.subplot2grid((100,100), (0,col_extra), colspan=col_len, rowspan=row_len)
j = 1
n = 14
run = run11
run.prune_data[n].plot_fit(j,linewidth=2)
axes(run.prune_data[n],j,x_low,x_high)
plt.axvline(-33.9686,ls='--',color='k',lw=1.2)
plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=12)

In [288]:
fig.savefig('S1 v3',dpi=1000)

In [306]:
#Fits for S1 and S2
height = 1
width = 1
row_extra = 10
row_len = (100-row_extra*(height))/height
col_extra = 5
col_len = (100-col_extra*(width))/width
fig = plt.figure(figsize = (2.8/(col_len/100.),1.69/(row_len/100.)))
x_low = -19.4859
x_high = -15.42069
plt.subplot2grid((100,100), (0,col_extra), colspan=col_len, rowspan=row_len)
j = 1
n = 2
run = run14
run.prune_data[n].plot_fit(j,linewidth=2)
axes(run.prune_data[n],j,x_low,x_high)
plt.axvline(-17.4533,ls='--',color='k',lw=1.2)
plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)

In [307]:
fig.savefig('S2 v1',dpi=1000)

In [18]:
height = 7
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi4_bounds = list(pi4_14_bounds)
pi4_bounds.extend(pi4_11_bounds)
pi4_bounds = np.array(pi4_bounds)

i = -1
for j in [3,9,10,6,1,0,2,4,5,8,11,12,13,14,17,18,15,7,16]:
    i+=1
    hold = pi4_bounds[j]
    if i == 17:
        i += 1
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    if i == 10:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    
    if i == 16:
        current = (5*(row_extra+row_len),1*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.022, 1.03),lw=1.2)
        continue
    if i == 19:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.75,-0.3)
    if i == 19:
        current = (6*(row_extra+row_len),1*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.022, 1.03),lw=1.2)
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    plt.title(j)
    

In [31]:
fig.savefig('025Pi Supplemental v6 Larger Windows',dpi=1000)

In [87]:
height = 5
width = 3
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi4_bounds = list(pi4_14_bounds)
pi4_bounds.extend(pi4_11_bounds)
pi4_bounds = np.array(pi4_bounds)

i = -1
for j in [3,9,10,6,1,0,2,4,5,8,11,12,14,15]:
    i+=1
    hold = pi4_bounds[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    if i == 7:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.75,-0.15)
    if i == 13:
        current = (4*(row_extra+row_len),1*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],2*col_len+col_extra,row_len,xy=(-0.022, 1.03),lw=1.2)
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)

    

In [88]:
fig.savefig('025Pi Supplemental v6 Pruned',dpi=1000)

In [75]:
height = 3
width = 3
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.85/(col_len/100.),1.0/(row_len/100.)))
pi4_bounds = list(pi4_14_bounds)
pi4_bounds.extend(pi4_11_bounds)
pi4_bounds = np.array(pi4_bounds)

out = list()

i = -1
for j in [3,9,10,6,1,0,2,4,5]:
    i+=1
    hold = pi4_bounds[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
  
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    #plt.title(j)
    out.append(newSE(*hold))
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
    if i == 7:
        plt.xlabel('$V_P$ $(mV)$',labelpad=5,size=12)

    

In [38]:
fig.savefig('025Pi Supplemental v7',dpi=1000)

In [63]:
np.save('Pruned 025 1_29_2019',out)

In [52]:
def newSE(run,se,fit_list,k,_a,_b):
    temp = list()
    j = fit_list[k][1]
    n = fit_list[k][0]
    for k in se[n][np.abs(se[n][:,2] - (run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi%2)<0.001]:
        temp.append(k)
    return temp


In [18]:
height = 2
width = 3
row_extra = 12
bot = 3
row_len = (100-row_extra*(height-1)-bot)/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [5,2,6,0,4,7]:
    hold = pi34_bounds[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-0.15,1.12)
    if i == 4:
        plt.xlabel('$V_P$ $(mV)$',labelpad=5,size=18)
    #out.append(newSE(*hold))    
    i += 1

In [35]:
fig.savefig('075Pi Supplemental v10',dpi=1000)

In [20]:
pi34_bounds = [[run14,se14,pi34_14,0,-13.17,-4.05],[soft14,se14,pi34_14,1,-28.02,-21.7],
               [run14,se14,pi34_14,2,-12.9,-4.06],[soft14,se14,pi34_14,3,-12.04,-4.05],
               [run11,se11,pi34_11,0,-41.1,-34.2],[soft11,se11,pi34_11,1,-40.5,-31.7],
               [run11,se11,pi34_11,2,-37.159,-29],[run11,se11,pi34_11,3,-39.48,-30.16],
               [soft11,se11,pi_11,0,-26.4479, -17.6429]]

In [29]:
height = 4
width = 2
row_extra = 6
bot = 3
row_len = (100-row_extra*(height-1)-bot)/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [0,2,4,5,-1,6,7]:
    hold = pi34_bounds[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    if i == 6:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len)+int((col_extra+col_len)/2))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    if i == 4:
        plt.ylabel('$R_D(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P(mV)$',labelpad=5,size=18)
        #ax = plt.gca()
        #ax.xaxis.set_label_coords(1.08,-0.23)
    #out.append(newSE(*hold))    
    i += 1

In [30]:
fig.savefig('52_34')

In [115]:
np.save('Pruned 075 2_1_2019',out)

In [32]:
height = 2
width = 2
row_extra = 12
bot = 5
row_len = (100-row_extra*(height-1)-bot)/height
col_extra = 9
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [6,3,0,4]:
    hold = pi2_bounds[j]
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-0.15,1.12)
    
    if i == 3:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.1,-0.25)
    out.append(newSE(*hold)) 
    i += 1


NameError: name 'pi2_bounds' is not defined

In [182]:
fig.savefig('05Pi Supplemental v7',dpi=1000)

In [183]:
np.save('Pruned 05 2_1_2019',out)

In [54]:
#Make modern holders
#1/2 first
pi2_new = list()
for j in [6,3,0,4]:
    pi2_new.append(pi2_bounds[j])
pi2_new[0][-2],pi2_new[0][-1] = -28.79,-18
pi2_new[2][-2],pi2_new[2][-1] = -22.22,-15.429
pi2_new[3][-2],pi2_new[3][-1] = -43.3619,-33.96

In [58]:
height = 2
width = 2
row_extra = 9
bot = 2
row_len = (100-row_extra*(height-1)-bot)/height
col_extra = 8
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in range(2):
    hold = pi2_new[j]
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-0.12,1.12)
    
    if i == 3:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.1,-0.17)
    out.append(newSE(*hold)) 
    i += 1


In [204]:
fig.savefig('05Pi Supplemental v8 longer',dpi=1000)

In [69]:
height = 1
width = 2
row_extra = 9
bot = 16
row_len = (100-row_extra*(height-1)-bot)/height
col_extra = 10
col_len = (100-col_extra*(width-1))/width

fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in range(2):
    hold = pi2_new[j]
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.03),lw=1.2)
    
    if i == 0:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=0,size=18)
    if i == 1:
        plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.1,-0.17)
    out.append(newSE(*hold)) 
    i += 1


In [70]:
fig.savefig('05Pi Supplemental v9',dpi=1000)

In [66]:
out

[[array([-24.66637   ,   5.5265    ,   0.47590564,   0.8836773 ])],
 [array([-30.623075  ,   5.534     ,   0.532419  ,   0.80487805]),
  array([-28.549115  ,   5.534     ,   0.532419  ,   0.80487805]),
  array([-28.18076   ,   5.534     ,   0.532419  ,   0.80487805]),
  array([-27.62813   ,   5.534     ,   0.532419  ,   0.80487805]),
  array([-27.450285  ,   5.534     ,   0.532419  ,   0.80487805])]]

In [68]:
np.save('Pruned 05 3_12_2019',out)

In [17]:
pi34_new = list()
for j in [5,2,6,0,4,7]:
    pi34_new.append(pi34_bounds[j])

In [163]:
plot(*hold)

In [171]:
plot(hold[0],hold[1],hold[2],hold[3],-22.22,-15.429)

In [207]:
hold = pi34_new[1]

In [17]:
pi4_new = list()

pi4_bounds = list(pi4_14_bounds)
pi4_bounds.extend(pi4_11_bounds)
pi4_bounds = np.array(pi4_bounds)


for j in [3,9,6,10,1,0,2,4,5]:
    pi4_new.append(pi4_bounds[j])

pi4_new[1][-2],pi4_new[1][-1] = -28.371,-19.6137
pi4_new[3][-2],pi4_new[3][-1] = -31.0697,-22.9274
pi4_new[4][-2],pi4_new[4][-1] = -22.15,-15.3109
pi4_new[5][-2],pi4_new[5][-1] = -13.4502,-6.9825
pi4_new[6][-2],pi4_new[6][-1] = -28.371,-19.4487

In [375]:
plot(*hold)

In [387]:
plot(hold[0],hold[1],hold[2],hold[3],-28.371,-19.4487)

In [385]:
hold = pi4_new[6]
print(hold[-4][hold[-3]])

[0, 3]


In [24]:
def p4_1(lw=2):
    n = 3
    global j
    j = 3
    init_plot(i14[n])
    i14[n].fits[4].plot_select('l','k',lw,-28,-21.7239)
    i14[n].fits[4].plot_select('l','k',lw,-21.7239,-21.15,'--')

    i14[n].fits[4].plot_select('r','r',lw,-24.56,-24.2582,'--')
    i14[n].fits[4].plot_select('r','r',lw,-24.2582,-23.9979)
    i14[n].fits[4].plot_select('r','r',lw,-23.9979,-23.67,'--')
    i14[n].fits[4].plot_select('r','r',lw,-22.1493,-21.615,'--')
    i14[n].fits[3].plot_select('l','r',lw,-21.615,-20.2475)
    i14[n].fits[3].plot_select('l','r',lw,-20.2475,-18.41,'--')

    #mid_color = '#00ffff'
    mid_color = '#00e5ff'
    
    i14[n].fits[3].plot_select('r',mid_color,lw,-20.51,-20.2475,'--')
    i14[n].fits[3].plot_select('r',mid_color,lw,-20.2475,-18.41)
    
    set_axis(i14[n],-26.8407,-19.0156)
    #plt.xticks([-25,-20])
    
    #plt.axvline(-20.2475,color='k',lw=1.2)
    for k in se14[n][np.abs(se14[n][:,2] - (2-i14[n].fits[4].delta_phi))<0.01]:
        plt.axvline(k[0],color='k',lw=1.2)
    for k in se14[n][np.abs(se14[n][:,2] - (2-i14[n].fits[3].delta_phi))<0.01]:
        plt.axvline(k[0],color='k',lw=1.2)
        break
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
p4_1()

In [25]:
#plt.figure()
def p4_0(lw = 2):
    
    global j
    n = 2

    j = 2
    init_plot(i14[n])
    i14[n].fits[2].plot_select('l','k',lw,-22.9,-19.6676)
    i14[n].fits[2].plot_select('l','k',lw,-19.6676,-19.23,'--')

    i14[n].fits[2].plot_select('r','r',lw,-20.2,-19.6676,'--')
    i14[n].fits[2].plot_select('r','r',lw,-19.6676,-18.6335)
    i14[n].fits[2].plot_select('r','r',lw,-18.6335,-18.223,'--')
    i14[n].fits[2].plot_select('r','r',lw,-18.223,-17.8369)
    i14[n].fits[1].plot_select('l','r',lw,-17.8369,-17.7601)
    i14[n].fits[1].plot_select('l','r',lw,-17.7601,-17.4343,'--')
    i14[n].fits[1].plot_select('l','r',lw,-17.343,-12.572)

    #mid_color = '#00ffff'
    mid_color = '#00e5ff'

    i14[n].fits[1].plot_select('r',mid_color,lw,-18.99,-18.6335,'--')
    i14[n].fits[1].plot_select('r',mid_color,lw,-18.6335,-18.223)
    i14[n].fits[1].plot_select('r',mid_color,lw,-18.223,-17.7601,'--')
    i14[n].fits[1].plot_select('r',mid_color,lw,-17.7601,-17.4343)
    i14[n].fits[1].plot_select('r',mid_color,lw,-17.343,-17.133,'--')

    set_axis(i14[n],-21.6517,-14.6178)
    #plt.xticks([-20,-15])

    plt.axvline(-19.4858,color='k',lw=1.2)
    plt.axvline(-18.62333,color='k',lw=1.2)
    plt.axvline(-18.2212,color='k',lw=1.2)
    plt.axvline(-17.759945,color='k',lw=1.2)
    plt.axvline(-17.42249,color='k',lw=1.2)
    
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
#p4_0()

In [389]:
height = 3
width = 3
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in range(9):
    hold = pi4_new[j]
    if j == 7 or j == 8:
        continue
    if i == 0:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        p4_0()
        i += 2
        continue
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        p4_1()
        i += 2
        continue    
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.05, 1.03),lw=1.2)
    
    if i == 3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)

    
    if i == 7:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
   
    out.append(newSE(*hold)) 
    i += 1


In [391]:
fig.savefig('025Pi Supplemental v8',dpi=1000)

In [26]:
def p4_2(lw=2):
    n = 8
    global j
    j = 0
    init_plot(i14[n])
    i14[n].fits[0].plot_select('l','k',lw,-31.0697,-30.9762)
    i14[n].fits[0].plot_select('l','k',lw,-30.9762,-30.1442,'--')
    i14[n].fits[0].plot_select('l','k',lw,-30.1442,-28.8462)
    i14[n].fits[0].plot_select('l','k',lw,-28.8462,-27.5842,'--')
    i14[n].fits[0].plot_select('l','k',lw,-27.5842,-26.6193)
    i14[n].fits[0].plot_select('l','k',lw,-26.6193,-26.2713,'--')
    
    i14[n].fits[0].plot_select('r','r',lw,-31.0697,-30.9762,'--')
    i14[n].fits[0].plot_select('r','r',lw,-30.9762,-30.1442)
    i14[n].fits[0].plot_select('r','r',lw,-30.1442,-28.8462,'--')
    i14[n].fits[0].plot_select('r','r',lw,-28.8462,-27.5842)
    i14[n].fits[0].plot_select('r','r',lw,-27.5842,-26.6193,'--')
    i14[n].fits[0].plot_select('r','r',lw,-26.6193,-25.1515)
    
    i14[n].fits[0].plot_select('r','r',lw,-25.1515,-25.0052,'--')
    i14[n].fits[0].plot_select('r','r',lw,-25.0052,-23.439)
    
    #mid_color = '#00ffff'
    mid_color = '#00e5ff'
    
    i14[n].fits[3].plot_select('l','r',lw,-23.439,-21.113)
    i14[n].fits[3].plot_select('l','r',lw,-21.113,-19.5408,'--')
    
    i14[n].fits[3].plot_select('l','r',lw,-19.5408,-19.3095)
    i14[n].fits[3].plot_select('l','r',lw,-19.3095,-19.0725,'--')
    
    i14[n].fits[3].plot_select('r',mid_color,lw,-25.6484,-25.1515,'--')
    i14[n].fits[3].plot_select('r',mid_color,lw,-25.1515,-25.0052)
    i14[n].fits[3].plot_select('r',mid_color,lw,-25.0052,-24.6198,'--')
    
    i14[n].fits[3].plot_select('r',mid_color,lw,-21.5354,-21.113,'--')
    i14[n].fits[3].plot_select('r',mid_color,lw,-21.113,-19.5408)
    i14[n].fits[3].plot_select('r',mid_color,lw,-19.5408,-19.3095,'--')
    i14[n].fits[3].plot_select('r',mid_color,lw,-19.3095,-18.975)
    
    set_axis(i14[n],-31.0697,-18.975)
    
    for k in se14[n][np.abs(se14[n][:,2] - (2-i14[n].fits[0].delta_phi))<0.01]:
        plt.axvline(k[0],color='k',lw=1.2)
    for k in se14[n][np.abs(se14[n][:,2] - (2- i14[n].fits[3].delta_phi))<0.01][0:4]:
        plt.axvline(k[0],color='k',lw=1.2)
    plt.axvline(-19.3095,color='k',lw=1.2)
    
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
p4_2()

In [464]:
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (2*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_0()
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i += 1
        continue
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_1()
        i+=1
        continue    
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=1.2)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [463]:
fig.savefig('025Pi Supplemental v9',dpi=1000)

In [473]:
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (2*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
     
    if i == 0:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_2()
        i+=1
        continue  
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_0()
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i += 1
        continue
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_1()
        i+=1
        continue    
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=1.2)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [461]:
fig.savefig('025Pi Supplemental v10',dpi=1000)

In [27]:
def p4_3(lw=2):
    n = 7
    global j
    j = 2
    init_plot(i14[n])
    i14[n].fits[j].plot_select('l','k',lw,-28.371,-21.9926)
    i14[n].fits[j].plot_select('l','k',lw,-21.9926,-21.0418,'--')
    i14[n].fits[j].plot_select('l','k',lw,-21.0418,-19.6137)
    i14[n].fits[j].plot_select('r','r',lw,-22.4051,-21.9926,'--')
    i14[n].fits[j].plot_select('r','r',lw,-21.9926,-21.0418)
    i14[n].fits[j].plot_select('r','r',lw,-21.0418,-20.6203,'--')
    
    
    set_axis(i14[n],-28.371,-19.6137)
    
    for k in se14[n][np.abs(se14[n][:,2] - (2-i14[n].fits[j].delta_phi))<0.01]:
        plt.axvline(k[0],color='k',lw=1.2)
    
    
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
p4_3()

In [28]:
def p4_4(lw=2):
    n = 0
    global j
    j = 2
    init_plot(i14[n])
    i14[n].fits[j].plot_select('l','k',lw,-22.15,-19.4337)
    i14[n].fits[j].plot_select('l','k',lw,-19.4337,-18.9451,'--')

    i14[n].fits[j].plot_select('r','r',lw,-19.8592,-19.4337,'--')
    i14[n].fits[j].plot_select('r','r',lw,-19.4337,-15.3109)
    
    
    set_axis(i14[n],-22.15,-15.3109)
    
    for k in se14[n][np.abs(se14[n][:,2] - (2-i14[n].fits[j].delta_phi))<0.01]:
        plt.axvline(k[0],color='k',lw=1.2)
    
    
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
p4_4()

In [29]:
def p4_5(lw=2):
    n = 0
    global j
    j = 0
    init_plot(i14[n])
    i14[n].fits[j].plot_select('l','k',lw,-13.4502,-8.44257)
    i14[n].fits[j].plot_select('l','k',lw,-8.44257,-8.1365,'--')

    i14[n].fits[j].plot_select('r','r',lw,-8.72,-8.44257,'--')
    i14[n].fits[j].plot_select('r','r',lw,-8.44257,-6.9825)
    
    
    set_axis(i14[n],-13.4502,-6.9825)
    
    for k in se14[n][np.abs(se14[n][:,2] - (2-i14[n].fits[j].delta_phi))<0.01]:
        plt.axvline(k[0],color='k',lw=1.2)
    
    
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
p4_5()

In [514]:
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (2*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
    
    if i == 0:
        p4_2()
    if i == 1:
        p4_3()
    if i == 2:
        p4_0()
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
    if i == 3:
        p4_1()  
    if i == 4:
        p4_4()
    if i == 5:
        p4_5()
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [486]:
fig.savefig('025Pi Supplemental v11',dpi=1000)

In [546]:
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
lw = 2.5
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_0(lw)
        i += 1
        continue
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_1(lw)
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue    
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=1.2,linew=lw)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [547]:
fig.savefig('025Pi Supplemental v12',dpi=1000)

In [37]:
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Middle from old style
lw = 2.5
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        q_0(lw,'#00e5ff')
        i += 1
        continue
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        q_1(lw,'#00e5ff')
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue    
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=1.2,linew=lw)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [38]:
fig.savefig('025Pi Supplemental v13',dpi=1000)

In [575]:
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Newish style and expanded 0
lw = 2.5
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    
    if i == 0:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_2(lw)
        i += 1
        continue
    

    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_0(lw)
        i += 1
        continue
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        p4_1(lw)
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue    
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=1.2,linew=lw)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [576]:
fig.savefig('025Pi Supplemental v14',dpi=1000)

In [30]:
def q_0(lw=2,linew=2,midcolor = '#00ff00'):
    n = 2
    global j
    j = 2
    init_plot(i14[n])
    i14[n].fits[2].plot_select('l','k',linew,-21.6517,-19.6676)
    i14[n].fits[2].plot_select('l','k',linew,-19.6676,-13.76,'--')

    i14[n].fits[2].plot_select('r','r',linew,-21.6517,-19.6676,'--')
    i14[n].fits[2].plot_select('r','r',linew,-19.6676,-18.6335)
    i14[n].fits[2].plot_select('r','r',linew,-18.6335,-18.223,'--')
    i14[n].fits[2].plot_select('r','r',linew,-18.223,-17.8369)
    i14[n].fits[1].plot_select('l','r',linew,-17.8369,-17.7601)
    i14[n].fits[1].plot_select('l','r',linew,-17.7601,-17.4343,'--')
    i14[n].fits[1].plot_select('l','r',linew,-17.343,-13.76)

    i14[n].fits[1].plot_select('r',midcolor,linew,-21.6517,-18.6335,'--')
    i14[n].fits[1].plot_select('r',midcolor,linew,-18.6335,-18.223)
    i14[n].fits[1].plot_select('r',midcolor,linew,-18.223,-17.7601,'--')
    i14[n].fits[1].plot_select('r',midcolor,linew,-17.7601,-17.4343)
    i14[n].fits[1].plot_select('r',midcolor,linew,-17.343,-13.76,'--')

    set_axis(i14[n],-21.6517,-14.6178)
    plt.xticks([-20,-15])
    
    plt.axvline(-19.4858,color='k',lw=lw)
    plt.axvline(-18.62333,color='k',lw=lw)
    plt.axvline(-18.2212,color='k',lw=lw)
    plt.axvline(-17.759945,color='k',lw=lw)
    plt.axvline(-17.42249,color='k',lw=lw)
    
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")

In [31]:
def q_1(lw=2,midcolor = '#00ffff'):
    n = 3
    global j
    j = 3
    init_plot(i14[n])
    i14[n].fits[4].plot_select('l','k',lw,-26.8855,-21.615)
    i14[n].fits[4].plot_select('l','k',lw,-21.615,-18.41,'--')

    i14[n].fits[4].plot_select('r','r',lw,-26.8855,-21.615,'--')
    i14[n].fits[3].plot_select('l','r',lw,-21.615,-20.2475)
    i14[n].fits[3].plot_select('l','r',lw,-20.2475,-18.41,'--')

    i14[n].fits[3].plot_select('r',midcolor,lw,-26.8855,-20.2475,'--')
    i14[n].fits[3].plot_select('r',midcolor,lw,-20.2475,-18.41)
    
    set_axis(i14[n],-26.8855,-19.0156)
    plt.xticks([-25,-20])
    
    plt.axvline(-20.2475,color='k',lw=1.2)
    for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
        plt.axvline(k[0],color='k',lw=1.2)
        
    xy=(-0.023, 1.02)
    plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")

In [55]:
#Version 15
#Change a few colors for multislip fits
#Thicker slip line markers
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Middle from old style
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','k',linew,-21.6517,-14.6178)

        i14[n].fits[2].plot_select('r','r',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','r',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','#00e5ff',linew,-21.6517,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','k',linew,-26.8855,-19.0156)

        i14[n].fits[4].plot_select('r','r',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','r',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','#00e5ff',linew,-26.8855,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue    
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [56]:
fig.savefig('025Pi Supplemental v15a',dpi=1000)

In [59]:
#Version 15b
#Brown line #663300
#Change a few colors for multislip fits
#Thicker slip line markers
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Middle from old style
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.7239)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-26.8855,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.5254)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-21.6517,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [61]:
fig.savefig('025Pi Supplemental v15b',dpi=1000)

In [69]:
#Version 15b
#Brown line #663300
#Change a few colors for multislip fits
#Thicker slip line markers
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Middle from old style
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.7239)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-21.7239,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.4861)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.4861,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [70]:
fig.savefig('025Pi Supplemental v15c',dpi=1000)

In [71]:
#Version 15b
#Brown line #663300
#Change a few colors for multislip fits
#Thicker slip line markers
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Middle from old style
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#00e5ff',linew,-26.8855,-21.7239)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-21.7239,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#00e5ff',linew,-21.6517,-19.4861)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.4861,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [72]:
fig.savefig('025Pi Supplemental v15d',dpi=1000)

In [84]:
#Version 15b
#Brown line #663300
#Change a few colors for multislip fits
#Thicker slip line markers
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Middle from old style
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]

    
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.7239)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-21.7239,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.4861)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.4861,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [85]:
fig.savefig('025Pi Supplemental v16',dpi=1000)

In [20]:
def newSE_count(run,se,n,j):
    temp = list()
    for k in se[n][np.abs(se[n][:,2] - (run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi%2)<0.001]:
        temp.append(k)
    return temp


In [34]:
#Version 15b
#Brown line #663300
#Change a few colors for multislip fits
#Thicker slip line markers
#Less period for data
#Darker mid color
#Fatter aspect ratio
#Old style most fits
#Thicker plot lines
#Middle from old style
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    
    
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        continue
    
    if i == 5:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.04,-0.17)
           
    out.append(newSE(*hold)) 
    i += 1


In [98]:
fig.savefig('025Pi Supplemental v17',dpi=1000)

In [132]:
temp = list()
for i in out:
    for j in i:
        temp.append(j)
        
temp = np.array(temp)

In [141]:
plt.hist(temp[:,2],bins = np.arange(0,0.5,0.03))
plt.xlabel('$\Delta \phi$',fontsize = 'large')

In [125]:
out = np.array(out)

In [129]:
plt.figure()
def sweep_0(lw = 2.0, linew = 2.5):
    global j
    n = 0
    j = 2

     #fit lines

    #init_plot(i14[n]) #v1
    plt.plot(d140[0],d140[1]) #v2

    i14[n].fits[3].plot_select('l','k',linew,-30,-21.51)

    i14[n].fits[2].plot_select('l','r',linew,-24.3588,-17.22)


    i14[n].fits[2].plot_select('r','k',linew,-20.542,-18.675)
    i14[n].fits[1].plot_select('l','k',linew,-18.675,-10.93)

    i14[n].fits[1].plot_select('r','r',linew,-17.22,-6.98202)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-15,-8.417)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-8.417,-6.98202,'--')


    i14[n].fits[0].plot_select('r','k',linew,-9.2448,-6.98202)

    for k in se14[n]:
        plt.axvline(k[0],color='k',lw=lw)

    set_axis(i14[n],-28,-6.98202)
sweep_0()

In [74]:
plt.savefig('025Pi 5845T v1',dpi=1000)

In [111]:
plt.figure()

n = 0
j=2

lw = 2.0 #slip lines
linew = 2.5 #fit lines

#init_plot(i14[n]) #v1
plt.plot(d140[0],d140[1]) #v2

#i14[n].fits[0].phase_l -= 0.002

i14[n].fits[0].plot(False)

i14[n].fits[1].plot(False)


In [130]:
#OPTION 1
plt.figure()
def sweep_1(lw = 2.0, linew = 2.5):
    global j
    n = 0
    j=2

    lw = 2.0 #slip lines
    linew = 2.5 #fit lines

    #init_plot(i14[n]) #v1
    plt.plot(d140[0],d140[1]) #v2

    i14[n].fits[3].plot_select('l','k',linew,-28,-21.51)

    i14[n].fits[3].plot_select('r','r',linew,-28,-22.6491)

    i14[n].fits[2].plot_select('l','r',linew,-22.6491,-17.22)


    i14[n].fits[2].plot_select('r','k',linew,-20.542,-18.675)
    i14[n].fits[1].plot_select('l','k',linew,-18.675,-10.93)

    i14[n].fits[1].plot_select('r','r',linew,-17.22,-6.98202)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-15,-8.417)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-8.417,-6.98202,'--')


    i14[n].fits[0].plot_select('r','k',linew,-9.2448,-6.98202)

    for k in se14[n]:
        plt.axvline(k[0],color='k',lw=lw)

    set_axis(i14[n],-28,-6.98202)


In [131]:
#OPTION 2

plt.figure()
def sweep_2(lw = 2.0, linew = 2.5):
    global j
    n = 0
    j=2

    lw = 2.0 #slip lines
    linew = 2.5 #fit lines

    #init_plot(i14[n]) #v1
    plt.plot(d140[0],d140[1]) #v2

    i14[n].fits[3].plot_select('l','k',linew,-28,-20.8)

    i14[n].fits[3].plot_select('r','r',linew,-28,-19.8804)

    i14[n].fits[2].plot_select('l','r',linew,-19.8804,-17.22)


    i14[n].fits[2].plot_select('r','k',linew,-20.8,-18.675)
    i14[n].fits[1].plot_select('l','k',linew,-18.675,-10.93)

    i14[n].fits[1].plot_select('r','r',linew,-17.22,-6.98202)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-15,-8.417)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-8.417,-6.98202,'--')


    i14[n].fits[0].plot_select('r','k',linew,-9.355,-6.98202)

    for k in se14[n]:
        plt.axvline(k[0],color='k',lw=lw)

    set_axis(i14[n],-28,-6.98202)


In [128]:
#OPTION 3
#Edit the phase a hair to line up

plt.figure()
def sweep_3(lw = 2.0, linew = 2.5):
    global j
    n = 0
    j=2

    lw = 2.0 #slip lines
    linew = 2.5 #fit lines

    #init_plot(i14[n]) #v1
    plt.plot(d140[0],d140[1]) #v2

    i14[n].fits[3].plot_select('l','k',linew,-28,-20.8)

    i14[n].fits[3].plot_select('r','r',linew,-28,-19.8804)

    i14[n].fits[2].plot_select('l','r',linew,-19.8804,-17.22)


    i14[n].fits[2].plot_select('r','k',linew,-20.8,-18.675)
    i14[n].fits[1].plot_select('l','k',linew,-18.675,-10.93)

    i14[n].fits[1].plot_select('r','r',linew,-17.22,-11.74)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-15,-8.417)
    #i14[n].fits[0].plot_select('l','#00e5ff',linew,-8.417,-6.98202,'--')

    i14[n].fits[0].plot_select('l','r',linew,-11.74,-6.98202)
    i14[n].fits[0].plot_select('r','k',linew,-10.9,-6.98202)

    for k in se14[n]:
        plt.axvline(k[0],color='k',lw=lw)

    set_axis(i14[n],-28,-6.98202)


In [134]:
#Version 19
#Now has extended sweep on bottom
#Rest same as 18
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        k = 0
        for i in [sweep_0,sweep_1,sweep_2,sweep_3]:
            plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
            i()
            plt.xlabel('$V_P$ $(mV)$',labelpad=0,size=12)
            fig.savefig('test_{}'.format(k))
            k += 1
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        continue
           
    out.append(newSE(*hold)) 
    i += 1


In [150]:
#Version 19
#Now has extended sweep on bottom
#Rest same as 18
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        sweep_3()
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        continue
           
    out.append(newSE(*hold)) 
    i += 1


In [141]:
fig.savefig('025Pi v18',dpi=1000)

In [155]:
#Version 19
#Now has extended sweep on bottom
#Rest same as 18
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        n = 0
        j=2

        lw = 2.0 #slip lines
        linew = 2.5 #fit lines

        #init_plot(i14[n]) #v1
        plt.plot(d140[0],d140[1]) #v2

        i14[n].fits[3].plot_select('l','k',linew,-28,-20.8)

        i14[n].fits[3].plot_select('r','r',linew,-28,-19.8804)

        i14[n].fits[2].plot_select('l','r',linew,-19.8804,-17.22)


        i14[n].fits[2].plot_select('r','k',linew,-20.8,-18.675)
        i14[n].fits[1].plot_select('l','k',linew,-18.675,-10.93)

        i14[n].fits[1].plot_select('r','r',linew,-17.22,-11.74)
        #i14[n].fits[0].plot_select('l','#00e5ff',linew,-15,-8.417)
        #i14[n].fits[0].plot_select('l','#00e5ff',linew,-8.417,-6.98202,'--')

        i14[n].fits[0].plot_select('l','r',linew,-11.74,-6.98202)
        i14[n].fits[0].plot_select('r','k',linew,-10.9,-6.98202)

        for k in se14[n]:
            plt.axvline(k[0],color='k',lw=lw)

        set_axis(i14[n],-24.84,-6.98202)
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        continue
           
    out.append(newSE(*hold)) 
    i += 1


In [156]:
fig.savefig('025Pi v18_b',dpi=1000)

In [178]:
#Version 19
#Now has extended sweep on bottom
#Rest same as 18
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        n = 0
        j=2

        lw = 2.0 #slip lines
        linew = 2.5 #fit lines

        #init_plot(i14[n]) #v1
        plt.plot(d140[0],d140[1]) #v2

        i14[n].fits[3].plot_select('l','k',linew,-24.942,-21.789)
        
        i14[n].fits[3].plot_select('r','r',linew,-23,-21.16)

        i14[n].fits[3].plot_select('r','k',linew,-20.28,-19.8804)
        i14[n].fits[2].plot_select('l','k',linew,-19.8804,-18.77)


        i14[n].fits[2].plot_select('r','r',linew,-19.88,-18.675)
        i14[n].fits[1].plot_select('l','r',linew,-18.675,-17.7)
        
        i14[n].fits[1].plot_select('l','k',linew,-16.59,-13.88)

        i14[n].fits[1].plot_select('r','r',linew,-15.48,-12.73)


        i14[n].fits[0].plot_select('l','k',linew,-10.42,-7.89)
        i14[n].fits[0].plot_select('r','r',linew,-9,-6.98202)

        for k in se14[n]:
            plt.axvline(k[0],color='k',lw=lw)

        set_axis(i14[n],-28,-6.98202)
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        continue
           
    out.append(newSE(*hold)) 
    i += 1


In [171]:
fig.savefig('025Pi v19',dpi=1000)

In [176]:
#Version 19
#Now has extended sweep on bottom
#Rest same as 18
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        5
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        n = 0
        j=2

        lw = 2.0 #slip lines
        linew = 2.5 #fit lines

        #init_plot(i14[n]) #v1
        plt.plot(d140[0],d140[1]) #v2

        i14[n].fits[3].plot_select('l','k',linew,-24.942,-21.789)
        
        i14[n].fits[3].plot_select('r','r',linew,-23,-21.16)

        i14[n].fits[3].plot_select('r','k',linew,-20.28,-19.8804)
        i14[n].fits[2].plot_select('l','k',linew,-19.8804,-18.77)


        i14[n].fits[2].plot_select('r','r',linew,-19.88,-18.675)
        i14[n].fits[1].plot_select('l','r',linew,-18.675,-17.7)
        
        i14[n].fits[1].plot_select('l','k',linew,-16.59,-13.88)

        i14[n].fits[1].plot_select('r','r',linew,-15.48,-12.73)


        i14[n].fits[0].plot_select('l','k',linew,-10.42,-7.89)
        i14[n].fits[0].plot_select('r','r',linew,-9,-6.98202)

        for k in se14[n]:
            plt.axvline(k[0],color='k',lw=lw)

        set_axis(i14[n],-24.84,-6.98202)
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        continue
           
    out.append(newSE(*hold)) 
    i += 1


In [175]:
fig.savefig('025Pi v19_b',dpi=1000)

In [184]:
#Version 19
#Now has extended sweep on bottom
#Rest same as 18
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 5
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=12)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.xticks([-20,-15])
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        n = 0
        j=2

        lw = 2.0 #slip lines
        linew = 2.5 #fit lines

        #init_plot(i14[n]) #v1
        plt.plot(d140[0],d140[1]) #v2

        i14[n].fits[3].plot_select('l','k',linew,-25.8592,-20.75)      
        i14[n].fits[3].plot_select('r','r',linew,-25.8592,-20.75)

        i14[n].fits[3].plot_select('r','k',linew,-20.4608,-19.8804)
        i14[n].fits[2].plot_select('l','k',linew,-19.8804,-17.2498)
        i14[n].fits[2].plot_select('r','r',linew,-20.4608,-18.675)
        i14[n].fits[1].plot_select('l','r',linew,-18.675,-17.2498)
        
        
        i14[n].fits[1].plot_select('l','k',linew,-16.8737,-11.0015)
        i14[n].fits[1].plot_select('r','r',linew,-16.8737,-11.0015)


        i14[n].fits[0].plot_select('l','k',linew,-10.5675,-6.98202)
        i14[n].fits[0].plot_select('r','r',linew,-10.5675,-6.98202)

        for k in se14[n]:
            plt.axvline(k[0],color='k',lw=lw)

        set_axis(i14[n],-26.0964,-6.98202)
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=12)
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        plt.xticks([-25,-20,-15,-10])
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        plt.xticks([-30,-25])
        continue
           
    if i == 1:
        plt.xticks([-25,-20])
    out.append(newSE(*hold)) 
    i += 1


In [185]:
fig.savefig('025Pi v20',dpi=1000)

In [332]:
#Version 21
#Now has extended sweep on bottom
#Rest same as 18
mpl.rcParams.update({'font.size': 11})
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 7
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=18)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.xticks([-20,-15])
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        n = 0
        j=2

        lw = 2.0 #slip lines
        linew = 2.5 #fit lines

        #init_plot(i14[n]) #v1
        plt.plot(d140[0],d140[1]) #v2

        i14[n].fits[3].plot_select('l','k',linew,-25.8592,-20.75)      
        i14[n].fits[3].plot_select('r','r',linew,-25.8592,-20.75)

        i14[n].fits[3].plot_select('r','k',linew,-20.4608,-19.8804)
        i14[n].fits[2].plot_select('l','k',linew,-19.8804,-17.2498)
        i14[n].fits[2].plot_select('r','r',linew,-20.4608,-18.675)
        i14[n].fits[1].plot_select('l','r',linew,-18.675,-17.2498)
        
        
        i14[n].fits[1].plot_select('l','k',linew,-16.8737,-11.0015)
        i14[n].fits[1].plot_select('r','r',linew,-16.8737,-11.0015)


        i14[n].fits[0].plot_select('l','k',linew,-10.5675,-6.98202)
        i14[n].fits[0].plot_select('r','r',linew,-10.5675,-6.98202)

        for k in se14[n]:
            plt.axvline(k[0],color='k',lw=lw)

        set_axis(i14[n],-26.0964,-6.98202)
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        plt.xticks([-25,-20,-15,-10])
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        plt.xticks([-30,-25])
        continue
           
    if i == 1:
        plt.xticks([-25,-20])
    out.append(newSE(*hold)) 
    i += 1


In [333]:
fig.savefig('025Pi v21',dpi=1000)

In [31]:
#Version 22
#Now has extended sweep on bottom
#Rest same as 18
mpl.rcParams.update({'font.size': 11})
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 7
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=18)
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.xticks([-20,-15])
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        i += 1
        continue
    
    
    if i == 4:
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        n = 0
        j=2

        lw = 2.0 #slip lines
        linew = 2.5 #fit lines

        init_plot(soft14.prune_data[n]) #v1
        #plt.plot(d140[0],d140[1]) #v2

        i14[n].fits[3].plot_select('l','k',linew,-25.8592,-20.75)      
        i14[n].fits[3].plot_select('r','r',linew,-25.8592,-20.75)

        i14[n].fits[3].plot_select('r','k',linew,-20.4608,-19.8804)
        i14[n].fits[2].plot_select('l','k',linew,-19.8804,-17.2498)
        i14[n].fits[2].plot_select('r','r',linew,-20.4608,-18.675)
        i14[n].fits[1].plot_select('l','r',linew,-18.675,-17.2498)
        
        
        i14[n].fits[1].plot_select('l','k',linew,-16.8737,-11.0015)
        i14[n].fits[1].plot_select('r','r',linew,-16.8737,-11.0015)


        i14[n].fits[0].plot_select('l','k',linew,-10.5675,-6.98202)
        i14[n].fits[0].plot_select('r','r',linew,-10.5675,-6.98202)

        for k in se14[n]:
            plt.axvline(k[0],color='k',lw=lw)

        set_axis(i14[n],-26.0964,-6.98202)
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        plt.xticks([-25,-20,-15,-10])
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        plt.xticks([-30,-25])
        continue
           
    if i == 1:
        plt.xticks([-25,-20])
    out.append(newSE(*hold)) 
    i += 1


In [336]:
fig.savefig('025Pi v22',dpi=1000)

In [71]:
#Version 22
#Now has extended sweep on bottom
#Rest same as 18
mpl.rcParams.update({'font.size': 11})
xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 2
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 7
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
i = 0
o = 5
out = list()
for j in [3,1,0,2,4,5]:
    hold = pi4_new[j]
    current = (o/width*(row_extra+row_len),o%width*(col_extra+col_len))
    o -= 1

    
    if i == 2:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 3
        j = 3
        init_plot(i14[n])
        i14[n].fits[4].plot_select('l','#663300',linew,-26.8855,-21.1362)

        i14[n].fits[4].plot_select('r','k',linew,-26.8855,-21.615)
        i14[n].fits[3].plot_select('l','k',linew,-21.615,-19.0156)

        i14[n].fits[3].plot_select('r','r',linew,-22.5746,-19.0156)

        set_axis(i14[n],-26.8855,-19.0156)
        plt.xticks([-25,-20])

        plt.axvline(-20.2475,color='k',lw=lw)
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[4].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,3,3)
        temp.append(newSE_count(run14,se14,3,4)[0])
        out.append(temp)

        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        
        i+=1
        continue
    
    if i == 3:
        plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)
        n = 2
        j = 2
        init_plot(i14[n])
        i14[n].fits[2].plot_select('l','#663300',linew,-21.6517,-19.2432)

        i14[n].fits[2].plot_select('r','k',linew,-21.6517,-17.8369)
        i14[n].fits[1].plot_select('l','k',linew,-17.8369,-14.6178)
        
        i14[n].fits[1].plot_select('r','r',linew,-19.839,-14.6178)
        set_axis(i14[n],-21.6517,-14.6178)
        
        #As the fits are all very close in delta_phi, this captures both fits
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[2].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        
        temp = newSE_count(run14,se14,2,2)
        temp.extend(newSE_count(run14,se14,2,1))
        out.append(temp)
        
        plt.xticks([-20,-15])
        
        plt.annotate('{:.4f} T'.format(i14[n].b_val), xy, xycoords="axes fraction")
        
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=18)
        
        i += 1
        continue
    
    
    if i == 4:
        current = (0,0)
        plt.subplot2grid((100,100), current, colspan=2*col_len+col_extra, rowspan=row_len)
        n = 0
        j = 2

        lw = 2.0 #slip lines
        linew = 2.5 #fit lines

        init_plot(soft14.prune_data[n]) #v1
        #plt.plot(d140[0],d140[1]) #v2

        i14[n].fits[3].plot_select('l','k',linew,-25.8592,-20.75)      
        i14[n].fits[3].plot_select('r','r',linew,-25.8592,-20.75)

        i14[n].fits[3].plot_select('r','k',linew,-20.4608,-19.8804)
        i14[n].fits[2].plot_select('l','k',linew,-19.8804,-17.2498)
        i14[n].fits[2].plot_select('r','r',linew,-20.4608,-18.675)
        i14[n].fits[1].plot_select('l','r',linew,-18.675,-17.2498)
        
        
        i14[n].fits[1].plot_select('l','k',linew,-16.8737,-11.0015)
        i14[n].fits[1].plot_select('r','r',linew,-16.8737,-11.0015)


        i14[n].fits[0].plot_select('l','k',linew,-10.5675,-6.98202)
        i14[n].fits[0].plot_select('r','r',linew,-10.5675,-6.98202)

        for k in se14[n]:
            plt.axvline(k[0],color='k',lw=lw)

        set_axis(i14[n],-26.0964,-6.98202)
        
        plt.annotate('{:.4f} T'.format(i14[0].b_val),(-0.01, 1.02), xycoords="axes fraction")
        
        
        
        temp = [se14[0][0]]
        temp.extend(newSE_count(run14,se14,0,1))
        temp.extend(newSE_count(run14,se14,0,2))
        temp.extend(newSE_count(run14,se14,0,3))
        out.append(temp)
        
        plt.xticks([-25,-20,-15,-10])
        break
    
    
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=(-0.023, 1.02),lw=lw,linew=linew)
    
    #Add extra lines for the nonplotted fit
    if i == 0:
        n = 8
        j = 3
        for k in se14[n][np.abs(se14[n][:,2] -  (2-i14[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        temp = newSE_count(run14,se14,8,0)
        temp.extend(newSE_count(run14,se14,8,3)[0:2])
        out.append(temp)
        i += 1
        plt.xticks([-30,-25])
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(-0.07,-0.22)
        continue
           
    if i == 1:
        plt.xticks([-25,-20])
        
    out.append(newSE(*hold)) 
    i += 1


In [45]:
fig.savefig('025Pi v22b',dpi=1000)

In [72]:
out

[[array([-30.9763    ,   5.5805    ,   0.35446229,   1.        ]),
  array([-30.11417   ,   5.5805    ,   0.35446229,   1.        ]),
  array([-28.8464    ,   5.5805    ,   0.35446229,   1.        ]),
  array([-26.6193    ,   5.5805    ,   0.35446229,   1.        ]),
  array([-27.5842    ,   5.5805    ,   0.35446229,   1.        ]),
  array([-25.1515    ,   5.5805    ,   0.25995879,   1.        ]),
  array([-25.007     ,   5.5805    ,   0.25995879,   1.        ])],
 [array([-21.99253   ,   5.581     ,   0.24561601,   1.        ]),
  array([-21.041845  ,   5.581     ,   0.24561601,   1.        ])],
 [array([-20.24955,   5.583  ,   0.25   ,   1.     ]),
  array([-19.38313,   5.583  ,   0.25   ,   1.     ]),
  array([-19.18627,   5.583  ,   0.25   ,   1.     ]),
  array([-24.267575  ,   5.583     ,   0.27320272,   1.        ])],
 [array([-19.4861    ,   5.5835    ,   0.27931602,   1.        ]),
  array([-18.62333   ,   5.5835    ,   0.28183099,   1.        ]),
  array([-18.2212    ,   5.5

In [73]:
np.save('Pruned 025 3_12_2019',out)